<a href="https://colab.research.google.com/github/comparativechrono/microscoPi/blob/main/mjpeg2mp4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Converting Raspberry Pi mjpeg to mp4 videos using ffmpeg

In [ ]:
! apt install ffmpeg

^C


In [ ]:
pip install ffmpeg-python

convert single file

In [ ]:
import ffmpeg

def convert_mjpeg_to_mp4(input_file, output_file):
    (
        ffmpeg
        .input(input_file, format='mjpeg')
        .output(output_file, vcodec='libx264')
        .run()
    )

input_file = 'heart6_2ir.mjpeg'
output_file = 'output6_2ir.mp4'

convert_mjpeg_to_mp4(input_file, output_file)


convert all files

In [ ]:
import ffmpeg
import os

def convert_mjpeg_to_mp4(input_file, output_file):
    (
        ffmpeg
        .input(input_file, format='mjpeg')
        .output(output_file, vcodec='libx264')
        .run()
    )

def convert_all_mjpegs_in_directory(directory):
    for filename in os.listdir(directory):
        if filename.endswith(".mjpeg"):
            input_file = os.path.join(directory, filename)
            output_file = os.path.join(directory, filename.replace(".mjpeg", ".mp4"))
            convert_mjpeg_to_mp4(input_file, output_file)
            print(f"Converted {input_file} to {output_file}")

directory = '/'  # Replace with your directory path
convert_all_mjpegs_in_directory(directory)


In this script:

*   Convert_all_mjpegs_in_directory function iterates over all files in the given directory.
*   It checks if the file ends with .mjpeg. If it does, the function constructs the full input and output file paths.
*   It then calls convert_mjpeg_to_mp4 for each .mjpeg file to perform the conversion.
*   After converting each file, it prints a message indicating the conversion is done.